<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/AICU_gpt4o_transribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ GPT-4o Transcribe
### 自分で作るオンライン会議文字起こし

Google Drive 上の Zoom/Meet 録画ファイル (mp4) を分割し、OpenAI GPT-4o-mini-transcribe を用いて文字起こしします。

このコードへのリンク [j.aicu.ai/gpt4oTransc](https://j.aicu.ai/gpt4oTransc)

解説は「[サクリ！AIツール: Zoom・Meet録画を自動で文字起こし！Colab × GPT-4oで作る自作爆速議事録ツール！](https://note.com/aicu/n/n239d9f47b1a1)」

- AICU Japan 株式会社 [https://note.com/aicu](https://note.com/aicu/n/n239d9f47b1a1)
-  (C) 2025 Akihiko SHIRAI - AICU Japan - 著作権は放棄していません (MIT License)

### 🔐 Google Colab シークレット設定マニュアル

このセルでは、Google Colab に保存された `OPENAI_API_KEY` を使って OpenAI API の接続テストを行います。

#### ✅ 事前準備
以下の手順で `OPENAI_API_KEY` を Colab に設定してください。

1. 画面左のサイドバーにある「鍵マーク（🔐）」をクリック（「環境設定」→「シークレットを管理」）
2. 「+ 新しいシークレットを追加」をクリック
3. 以下のように入力して「保存」：
   - **名前（Name）**: `OPENAI_API_KEY`
   - **値（Value）**: `sk-...` から始まるあなたのOpenAI APIキー

> ⚠️ これにより、コード内にハードコードせずに安全にAPIキーを扱うことができます。

---

設定後にこのセルを実行することで、APIキーの有効性や利用可能なモデル一覧が確認できます。

In [ ]:
# @title
# ✅ セットアップ（初回のみ）
!pip install openai ffmpeg-python --quiet
!apt-get install -y ffmpeg > /dev/null
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
# 🔍 OpenAI API キーの動作確認（Colab secrets使用）
import openai
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
if not api_key:
    raise ValueError("❌ 'OPENAI_API_KEY' が Colab のシークレットに設定されていません")

openai.api_key = api_key

try:
    models = openai.models.list()
    model_ids = [m.id for m in models.data]
    print("✅ OpenAI API キーは有効です")
    print("利用可能なモデル例:", ", ".join(model_ids[:5]), "...")
except Exception as e:
    print("❌ OpenAI API キーが無効または接続エラーです")
    print("エラー内容:", e)

## 🔧 Step 1: Zoom録画の設定と対象ファイルの決定

このセルでは以下の入力を行います：
- PROMPT: 文字起こし時にAIへ与える指示（例：「逐語的に」「要約せずに」など）
-	DIR: Google Drive上の録画フォルダパス（例：/content/drive/MyDrive/Meet Recordings）最後のスラッシュは不要です
- FILENAME: 対象となるファイル名（拡張子 .mp4 は省略可）
- MOVE_TO_DONE: 処理後に元の .mp4 ファイルを done/ サブフォルダへ移動（デフォルト：True）
- KEEP_MP3: 変換後の .mp3 を残すか（デフォルト：True）
-	SEGMENT_SECONDS: 音声分割の1チャンクの長さ（秒単位、デフォルト：600）

🎯 ファイル名が省略された場合は、フォルダ内の最初の .mp4 または拡張子なしファイルが自動的に選ばれます。

📌 ファイルが存在しない場合：
	1.	.mp4 拡張子を補って再確認
	2.	それでも見つからなければ FileNotFoundError を出して処理を停止します

In [ ]:
# @title 📋 動画Transcribe(文字起こし)の基本設定
PROMPT = "これはMeet会議の議事録です。逐語的にテキストを起こしてください。できるだけ日本語のニュアンスや発話者の意図を保持しつつ、聞き間違いのないよう丁寧に書き起こしてください。"  # @param {type:"string"}
DIR = "/content/drive/MyDrive/Meet Recordings"  # @param {type:"string"}
FILENAME = ""  # @param {type:"string"}
MOVE_DONE = False  # @param {type:"boolean"}
KEEP_MP3 = False  # @param {type:"boolean"}
SEGMENT_SECONDS = 600  # @param {type:"number"}

from pathlib import Path

# 📂 入力パスの処理
DIR = Path(DIR).expanduser()
PROMPT = PROMPT.strip()
FILENAME = FILENAME.strip()

In [ ]:
# @title
# 🧠 書き起こし実行コード
import os
import ffmpeg
import time
from pathlib import Path
import openai
from google.colab import userdata

# 🔑 OpenAI APIキー取得（Colabシークレットから）
openai.api_key = userdata.get("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("❌ OPENAI_API_KEYが未設定です。Colabのシークレットで設定してください。")

# 📂 入力パス・パラメータ
DIR = Path(DIR).expanduser()
PROMPT = PROMPT.strip()
FILENAME = FILENAME.strip()

# 🎯 対象ファイル選定
files = sorted([f for f in DIR.iterdir() if f.suffix == '.mp4' or '.' not in f.name])
if not files:
    raise FileNotFoundError(f"{DIR} に .mp4 ファイルが見つかりません")

target_file_stem = FILENAME if FILENAME else files[0].stem
input_path = DIR / target_file_stem
if not input_path.exists():
    input_path = DIR / f"{target_file_stem}.mp4"
    if not input_path.exists():
        raise FileNotFoundError(f"対象ファイルが見つかりません: {input_path}")

print(f"🎯 処理対象: {input_path.name}")

# 🎞️ mp4 → mp3 変換
mp3_path = DIR / f"{input_path.stem}.mp3"
print(f"🎞️ mp3変換中: {input_path.name} → {mp3_path.name}")
ffmpeg.input(str(input_path)).output(
    str(mp3_path), vn=None, ar=44100, ac=2, **{'b:a': '128k'}
).overwrite_output().run()

# ⏱️ 音声長取得
duration = float(ffmpeg.probe(str(mp3_path))['format']['duration'])
print(f"⏱️ 長さ: {duration:.1f} 秒")

# ✂️ 分割
segment_dir = DIR / "mp3segment"
segment_dir.mkdir(exist_ok=True)
print(f"✂️ {SEGMENT_SECONDS}秒ごとに分割中...")
seg_fmt = segment_dir / f"{input_path.stem}_%03d.mp3"
ffmpeg.input(str(mp3_path)).output(
    str(seg_fmt), f='segment', segment_time=SEGMENT_SECONDS, c='copy'
).overwrite_output().run()

# 🎧 セグメント取得
segments = sorted(segment_dir.glob("*.mp3"))

# 📝 書き起こし
output_txt = DIR / f"{input_path.stem}.txt"
with output_txt.open("w", encoding="utf-8") as out:
    out.write(f"=== GPT-4o-mini Transcribe: {input_path.name} ===\n\n")
    out.flush()

    for i, seg_path in enumerate(segments):
        print(f"🎙️ Segment {i+1}/{len(segments)}: {seg_path.name}")
        with seg_path.open("rb") as f:
            try:
                t0 = time.time()
                result = openai.audio.transcriptions.create(
                    model="gpt-4o-mini-transcribe",
                    file=f,
                    language="ja",
                    prompt=PROMPT,
                    temperature=0.2
                )
                dt = time.time() - t0
                text = result.text.strip()
                out.write(f"[{seg_path.name}] ({dt:.1f} 秒)\n{text}\n\n")
                out.flush()
            except Exception as e:
                msg = f"[{seg_path.name}] エラー: {e}\n\n"
                print(msg)
                out.write(msg)
                out.flush()

# 📁 完了処理
if MOVE_DONE:
    done_dir = DIR / "done"
    done_dir.mkdir(exist_ok=True)
    input_path.rename(done_dir / input_path.name)
    print(f"📦 移動完了: {done_dir / input_path.name}")

if not KEEP_MP3:
    print("🧹 mp3ファイルを削除します")
    try:
        if mp3_path.exists():
            mp3_path.unlink()
            print(f"🗑️ 削除: {mp3_path}")
        for f in segment_dir.glob("*.mp3"):
            f.unlink()
            print(f"🗑️ 削除: {f}")
        if not any(segment_dir.iterdir()):
            segment_dir.rmdir()
            print(f"🗑️ 削除: {segment_dir}")
    except Exception as e:
        print(f"⚠️ mp3削除に失敗: {e}")

print("✅ 書き起こし完了")

## MIT License

Copyright (c) 2025 Akihiko Shirai / AICU Inc.

本ソースコードはMITライセンスのもとで提供されます。
ただし、次の補足条項に同意した上でご利用ください：

- 本ソースコードは無保証です。動作の正確性や目的適合性は保証しません。
- ソースコードを改変して使用することは可能ですが、改変後の不具合・挙動について、著作者は一切のサポート責任を負いません。
- 再配布時には本ライセンス文書および著作権表示を残してください。

This software is provided under the MIT License:

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the “Software”), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, subject to the following conditions:

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

Additional Notes:

- This code is provided for educational and experimental purposes.
- The authors do not guarantee its functionality or fitness for any particular purpose.
- Support for modified versions will not be provided.